## First Round Сountry Сheck

[] - iOS, [] - Android
-----------------------
- [-],[-] Austria
- [-],[-] Belgium
- [-],[+] Croatia
- [+],[-] Czechia (Czech Republic)
- [+],[+] Denmark
- [+],[+] Estonia
- [+],[-] Finland
- [-],[-] France
- [-],[-] Germany
- [-],[+] Greece
- [+],[-] Hungary
- [-],[-] Italy
- [+],[-] Latvia
- [-],[+] Lithuania
- [-],[-] Luxembourg
- [+],[-] Malta
- [+],[+] Netherlands
- [-],[+] Poland
- [+],[-] Portugal
- [-],[-] Slovakia
- [+],[+] Slovenia
- [-],[+] Spain
- [-],[-] Sweden
- [-],[+] Bulgaria (from March 2024)
- [+],[-] Romania (from March 2024)

Non-EU Member States:
---
- [-],[-] Iceland
- [+],[-] Liechtenstein
- [-],[-] Norway
- [-],[+] Switzerland

Outside Schengen but with special status:
---
- [+],[+] Ireland 
- [+],[+] United Kingdom
- [-],[-] Cyprus 


North American countries:
- [+],[+] Canada
- [+],[+] United States
- [-],[+] Mexico

In [18]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import json
import os

load_dotenv()
api_key = os.getenv("API_KEY")

In [90]:
search = GoogleSearch({
    "engine": "google_jobs",
    "q": "Android developer", 
    "location": "Mexico",
    "api_key": api_key
  })
result = search.get_dict()

file_path = r"Test\Mexico1.json"  

# Saving JSON to a file
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

Version 1:
search = GoogleSearch({
    "engine": "google_jobs",
    "q": "Job title", 
    "location": "Country",
    "api_key": api_key
  })

In [91]:
import shutil

# Путь к папке "Data Collection/google_domain_dot_com"
base_dir = os.path.join(os.getcwd(), "google_domain_dot_com")
android_dir = os.path.join(base_dir, "Android")
ios_dir = os.path.join(base_dir, "iOS")

# Убедимся, что папки Android и iOS существуют
os.makedirs(android_dir, exist_ok=True)
os.makedirs(ios_dir, exist_ok=True)

# Перебираем файлы в папке google_domain_dot_com
for file_name in os.listdir(base_dir):
    # Полный путь к файлу
    file_path = os.path.join(base_dir, file_name)

    # Проверяем, является ли это файлом (а не папкой)
    if os.path.isfile(file_path):
        # Если файл соответствует шаблону для iOS
        if file_name.endswith(".json") and not file_name[:-5].endswith("1"):
            shutil.move(file_path, os.path.join(ios_dir, file_name))
        # Если файл соответствует шаблону для Android
        elif file_name.endswith(".json") and file_name[:-5].endswith("1"):
            shutil.move(file_path, os.path.join(android_dir, file_name))

print("Файлы успешно перемещены по папкам!")


Файлы успешно перемещены по папкам!
